# Another memory problem

Problem: Processes that are each too large to fit into memory
1. Overlays: Programmer manually divided the program into pieces, or overlays, overlay manager swaps overlays in and out during execution, only necessary pieces in memory at a given time
2. Paging: Portions of virtual address space in physical memory at one time, virtual address space divided into equal pieces, needed pieces are brought in from memory, virutal address space can be greater than available physical memory.

# Paging

1. Divides a process' virtual address space into fixed sized pages
2. Stores a copy of that address space on disk
3. Views the physical memory as a series of equal sized page frames
4. Moves the pages into frames in memory
5. Manages the pages in memory

# Pages

A process' virtual address space is partitioned into equal sized pages. A virtual address is a pair (page number, page offset). The virtual address is equivalent to max page offset * page number + page offset.

# Page frames

Physical memory is partitioned into equal sized page frames. A page size is equivalent to the frame size. A memory address is a pair (frame number, frame offset). The physical address is frame number * frame offset max + frame offset.

# Virtual to physical allocation policy

Pages map to frames, pages are contiguous in the virtual address space, but pages are arbitrarily located in physical memory, and not all pages are mapped at all times.  
Problem is how do we find addresses when pages are not allocated contiguously in physical memory?  
Solution: Page table keeps track of mapping of pages to page frames, mapping is invisible to process.

Virtual address translation steps:
1. Program gives virtual address to the CPU to translate
2. The MMU splits the virtual address into its page and offset numbers
3. Since the offset is the same in virtual and physical memory, it is sent along with no change.
4. The page number is translated into a frame number, look into page table to find if it exists, if it does, send the frame number. If it doesn't pull it from disk and send the grame number
5. Append the offset to the end of the frame number to get the full physical address.

Paging allows shared memory, memory useed by a process no longer needs to be contiguous, shared page may exist in different parts of the virtual address of each process, but map to the same physical address.

Only one page table per process, as a part of the the process state, this stops processes from corrupting each others' page tables and accessing each others' memory.  
Page table entry contents: flgas, frame number.  
There is a page table base register that holds the base address of each page table. A page number is added to the base register to get the register of the corresponding entry in the page table for that page.

Performance issue: Virtual memory references require two memory references, one to access page table entry, one to get data.

# Improving speed: TLB (Translation lookaside buffers)

TLB adds an extra step to getting a frame number form the page numner. The TLB holds recently used frame/page pairings. They have a high hit ratio, since they exploit locality. If it is a TLB hit, it takes one cycle/transaction.

Issues with space: two sources, data structure overhead, page table, fragmentation   
Page table can be very large.

# Dealing with large page tables - Multi level paging

Add additional levels of indirection to the page table by subdividing the page number into k parts, which creates a k depth tree of tables.   
Each entry in the first level points to the second leve, which points to the third level, which holds the bit/frame number information associated with the page table entry. It saves space, since you only need to allocate space for the page table levels you need.

Multilevel page tables work well for 32 bit addresses, don't work well for 64 bit, too many levels.  
Instead of making tables proportional to the size of the virtual address space, make them proportional to the size of the physical address space, one entry for each physical page

# Inverted page tables

Each frame associated with an entry, contains a residence bit, occupier, protection bits.  
As the CPU generates virtual addresses, we must search the entire page table for the physical page, and we can solve this by either caching it, or using the TLB to hold all heavily used pages, but the TLB is limited in size.  
We can use a hash table!  

# Hashed inverted table

Hash page numbers to find corresponding frame number, page frame number not explicitly stored, protection, dirty, used, resident bits also in entry.  
Hash functions takes in the PID and the page number, which is added to the page table base register to get the address of the right page table entry. Once at the right page table entry, the PID and page number saved in the entry are checked against the PID and page number, if the don't match, we need to swap pages, otherwise, send the frame number along.

# The Big Picture

CPU generates a virtual address, address is trnaslated into physical address, used to reference physical memory.  
1. CPU generates virtual address
2. Virtual address used to reference a virtually addressed cache, if there's a match, we are finished
3. Otherwise, we need to access the TLB, and if there is a match, we get the physical address to access the physically addressed cache.
4. If there is no match, we must go to the page table and find the frame number using the page table base register and the offser values gained from the page table.
5. If there is a match in this cache, we are done.
6. Otherwise, the physical address is used to access main memory.

A process' virtual address space is part of its context, code pages are stored on disk, data and stack pages are also stored on file, OS determines which portions of a process' virtual address space are mapped in memory at any time.

# Page faults

References to unmapped pages results in a page fault  
Steps to handle a page fault
1. Exception is thrown, process is blocked, kernel mode flipped, page fault handler called
2. OS starts read of unmapped page
3. OS maps missing page into memory
4. OS returns control to the faulting process, unblocking it